In [ ]:
#@title Importar Pacotes

!pip install cython==0.29.36 numpy
!pip install sparse_dot_topn_for_blocks --no-build-isolation
!pip install string_grouper --no-build-isolation
!pip install dbfread
!pip install xlsxwriter
!pip install unidecode
!pip install fuzzywuzzy

import pandas as pd
from string_grouper import match_strings, match_most_similar, \
	group_similar_strings, compute_pairwise_similarities, \
	StringGrouper
from fuzzywuzzy import fuzz
from dbfread import DBF
from google.colab import files
from itertools import groupby
from IPython.display import clear_output
from datetime import datetime
from datetime import date as dt
import numpy as np
from datetime import timedelta
from IPython.display import clear_output
import unidecode
from collections import defaultdict
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
file_path = 'desiredfile.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

In [ ]:
#@title LEITURA DO BANCO
from google.colab import drive

#Faz a ligação com o Drive.
drive.mount('/content/gdrive/', force_remount=True)

#Entra no diretório do drive (comando do BASH).
!cd /content/gdrive/

#Cria as variavéis para salvar o nome do arquivo a ser aberto e variavéis do loop de interface.
file_name = None
found = False
choose = None

#Loop da interface
while found == False:

  #Pede o inpute do nome do arquivo.
  file_name = str
  choose = None
  search = input('\nNome do Arquivo: ') + '*'

  #Comando bash para pesquisar o nome do arquivo no diretório e retornar arquivos encontrados para a variavél "search" como uma lista.
  search = !find . -type f -iname $search

  #Loop para presentar arquivos encontrados.
  print('Arquivos encontrados:')
  for i in range(len(search)):
    print(f'{i+1}: {search[i]} \n')

  #Caso nenhum arquivo seja encontrado o tamanho da lista == 0, levanta erro.
  if len(search) == 0:
    print("Arquivo não encontrado ou formato inválido(.xlsx ou .DBF)")
    print("Certifique-se de que o nome do arquivo não contenha espaços")

  #Opção de seleção do arquivo
  print("N: Pesquisar novamente")
  choose = input('\nSelecionar o arquivo: ')

  #Caso seleção não seja "N" seleciona o nome do arquivo na lista e encerra o loop com 'found = True'.
  if choose.upper() != 'n'.upper():
    choose = int(choose)-1
    file_name = search[choose]
    found = True
    clear_output()

#Testa a extensão do arquivo escolhido e faz a leitura a apropiada, caso não seja xlsx ou DBF levanta erro.
try:
  if file_name[-3:] == 'xlsx':
    print("Carregando...")
    df = pd.read_excel(file_name)
    print("###################CONCLUÍDO###################")
  elif file_name[-3:] == 'DBF':
    print("Carregando...")
    dbf = DBF(file_name ,encoding='ISO8859-1')
    df = pd.DataFrame(iter(dbf))
    clear_output()
    print("###################CONCLUÍDO###################")
  else:
    raise ValueError()
except:
  print("Arquivo não encontrado ou formato inválido(.xlsx ou .DBF)")
  print("Certifique-se de que o nome do arquivo não contenha espaços")



In [ ]:
#@title VISUALIZAR BANCO
df.head()

In [ ]:
#@title NOME DAS COLUNAS (OBRIGATÓRIOS: NOME, DATA E MÃE ou MUNICIPIO)

#Inicia variaveis para loops de interface.
check = False
check_up = False

#Loop para input do nome de coluna.
while not check:
  name = input('Nome da Coluna de NOMES: ').strip()

  #Tese para confirmar que o input obrigatório foi feito.
  if len(name) == 0:
    print('Nome obrigatório!')

  #Teste para verificar se o nome da coluna se encontra no dataframe
  elif name not in df.columns:
    clear_output()
    print(f'\"{name}\" não esta no dataframe, tente novamente.')

  #Encerra o loop caso input passe nos testes.
  else: check = True

check = False
while not check:
  date = input('Nome da Coluna de DATA DE NASCIMENTO: ').strip()
  if len(date) == 0:
    print('Data obrigatório!')
  elif date not in df.columns:
    clear_output()
    print(f'\"{date}\" não esta no dataframe, tente novamente.')
  else: check = True


#Loop de colunas opcionais.
while not check_up:
  check = False
  while not check:
    mother = input('Nome da Coluna de NOMES DA MÃE: ').strip()
    if mother == "":
      check = True
    else:
      if mother not in df.columns:
        clear_output()
        print(f'\"{mother}\" não esta no dataframe, tente novamente.')
      else: check = True

  check = False
  while not check:
    mun = input('Nome da Coluna de MUNÍCIPIO: ').strip()
    if mun == "":
      check = True
    else:
      if mun not in df.columns:
        clear_output()
        print(f'\"{mun}\" não esta no dataframe, tente novamente.')
      else: check = True

  #Teste para verificar se pelo menos uma das duas colunas foram preenchidas
  if (len(mother) == 0 and len(mun) == 0):
    raise TypeError('Mãe ou Data deve ser passado')
  else: check_up = True


#Tratamento de datas, em especial para o banco do SIM.
df[date] = df[date].apply(lambda x:  str(x).zfill(8) if len(str(x)) == 7 else str(x))
df[date] = df[date].apply(lambda x: (x[4:8] + '-' + x[2:4] + '-' + x[:2]) if len(str(x)) == 8 else str(x))
df[date] = pd.to_datetime(df[date],errors='coerce')
df[date] = df[date].apply(str)
df[date] = df[date].apply(lambda x: np.nan if x == 'NaT' else x )
df[date] = df[date].apply(str)

Nome da Coluna de NOMES: NM_PACIENT	
Nome da Coluna de DATA DE NASCIMENTO: DT_NASC
Nome da Coluna de NOMES DA MÃE: 
Nome da Coluna de MUNÍCIPIO: ID_MN_RESI


In [ ]:
#Threshold do primeiro agrupamento de similaridade dos nomes.
min_similarity = 0.85

In [ ]:
#@title Função principal
# dicionario de id municipio (dict_id_mun)
dict_id_mun = {'430003': 'Aceguá', '430005': 'Água Santa', '430010': 'Agudo', '430020': 'Ajuricaba', '430030': 'Alecrim', '430040': 'Alegrete', '430045': 'Alegria', '430047': 'Almirante Tamandaré do Sul', '430050': 'Alpestre', '430055': 'Alto Alegre', '430057': 'Alto Feliz', '430060': 'Alvorada', '430063': 'Amaral Ferrador', '430064': 'Ametista do Sul', '430066': 'André da Rocha', '430070': 'Anta Gorda', '430080': 'Antônio Prado', '430085': 'Arambaré', '430087': 'Araricá', '430090': 'Aratiba', '430100': 'Arroio do Meio', '430105': 'Arroio do Sal', '430107': 'Arroio do Padre', '430110': 'Arroio dos Ratos', '430120': 'Arroio do Tigre', '430130': 'Arroio Grande', '430140': 'Arvorezinha', '430150': 'Augusto Pestana', '430155': 'Áurea', '430160': 'Bagé', '430163': 'Balneário Pinhal', '430165': 'Barão', '430170': 'Barão de Cotegipe', '430175': 'Barão do Triunfo', '430180': 'Barracão', '430185': 'Barra do Guarita', '430187': 'Barra do Quaraí', '430190': 'Barra do Ribeiro', '430192': 'Barra do Rio Azul', '430195': 'Barra Funda', '430200': 'Barros Cassal', '430205': 'Benjamin Constant do Sul', '430210': 'Bento Gonçalves', '430215': 'Boa Vista das Missões', '430220': 'Boa Vista do Buricá', '430222': 'Boa Vista do Cadeado', '430223': 'Boa Vista do Incra', '430225': 'Boa Vista do Sul', '430230': 'Bom Jesus', '430235': 'Bom Princípio', '430237': 'Bom Progresso', '430240': 'Bom Retiro do Sul', '430245': 'Boqueirão do Leão', '430250': 'Bossoroca', '430258': 'Bozano', '430260': 'Braga', '430265': 'Brochier', '430270': 'Butiá', '430280': 'Caçapava do Sul', '430290': 'Cacequi', '430300': 'Cachoeira do Sul', '430310': 'Cachoeirinha', '430320': 'Cacique Doble', '430330': 'Caibaté', '430340': 'Caiçara', '430350': 'Camaquã', '430355': 'Camargo', '430360': 'Cambará do Sul', '430367': 'Campestre da Serra', '430370': 'Campina das Missões', '430380': 'Campinas do Sul', '430390': 'Campo Bom', '430400': 'Campo Novo', '430410': 'Campos Borges', '430420': 'Candelária', '430430': 'Cândido Godói', '430435': 'Candiota', '430440': 'Canela', '430450': 'Canguçu', '430460': 'Canoas', '430461': 'Canudos do Vale', '430462': 'Capão Bonito do Sul', '430463': 'Capão da Canoa', '430465': 'Capão do Cipó', '430466': 'Capão do Leão', '430467': 'Capivari do Sul', '430468': 'Capela de Santana', '430469': 'Capitão', '430470': 'Carazinho', '430471': 'Caraá', '430480': 'Carlos Barbosa', '430485': 'Carlos Gomes', '430490': 'Casca', '430495': 'Caseiros', '430500': 'Catuípe', '430510': 'Caxias do Sul', '430511': 'Centenário', '430512': 'Cerrito', '430513': 'Cerro Branco', '430515': 'Cerro Grande', '430517': 'Cerro Grande do Sul', '430520': 'Cerro Largo', '430530': 'Chapada', '430535': 'Charqueadas', '430537': 'Charrua', '430540': 'Chiapeta', '430543': 'Chuí', '430544': 'Chuvisca', '430545': 'Cidreira', '430550': 'Ciríaco', '430558': 'Colinas', '430560': 'Colorado', '430570': 'Condor', '430580': 'Constantina', '430583': 'Coqueiro Baixo', '430585': 'Coqueiros do Sul', '430587': 'Coronel Barros', '430590': 'Coronel Bicaco', '430593': 'Coronel Pilar', '430595': 'Cotiporã', '430597': 'Coxilha', '430600': 'Crissiumal', '430605': 'Cristal', '430607': 'Cristal do Sul', '430610': 'Cruz Alta', '430613': 'Cruzaltense', '430620': 'Cruzeiro do Sul', '430630': 'David Canabarro', '430632': 'Derrubadas', '430635': 'Dezesseis de Novembro', '430637': 'Dilermando de Aguiar', '430640': 'Dois Irmãos', '430642': 'Dois Irmãos das Missões', '430645': 'Dois Lajeados', '430650': 'Dom Feliciano', '430655': 'Dom Pedro de Alcântara', '430660': 'Dom Pedrito', '430670': 'Dona Francisca', '430673': 'Doutor Maurício Cardoso', '430675': 'Doutor Ricardo', '430676': 'Eldorado do Sul', '430680': 'Encantado', '430690': 'Encruzilhada do Sul', '430692': 'Engenho Velho', '430693': 'Entre-Ijuís', '430695': 'Entre Rios do Sul', '430697': 'Erebango', '430700': 'Erechim', '430705': 'Ernestina', '430710': 'Herval', '430720': 'Erval Grande', '430730': 'Erval Seco', '430740': 'Esmeralda', '430745': 'Esperança do Sul', '430750': 'Espumoso', '430755': 'Estação', '430760': 'Estância Velha', '430770': 'Esteio', '430780': 'Estrela', '430781': 'Estrela Velha', '430783': 'Eugênio de Castro', '430786': 'Fagundes Varela', '430790': 'Farroupilha', '430800': 'Faxinal do Soturno', '430805': 'Faxinalzinho', '430807': 'Fazenda Vilanova', '430810': 'Feliz', '430820': 'Flores da Cunha', '430825': 'Floriano Peixoto', '430830': 'Fontoura Xavier', '430840': 'Formigueiro', '430843': 'Forquetinha', '430845': 'Fortaleza dos Valos', '430850': 'Frederico Westphalen', '430860': 'Garibaldi', '430865': 'Garruchos', '430870': 'Gaurama', '430880': 'General Câmara', '430885': 'Gentil', '430890': 'Getúlio Vargas', '430900': 'Giruá', '430905': 'Glorinha', '430910': 'Gramado', '430912': 'Gramado dos Loureiros', '430915': 'Gramado Xavier', '430920': 'Gravataí', '430925': 'Guabiju', '430930': 'Guaíba', '430940': 'Guaporé', '430950': 'Guarani das Missões', '430955': 'Harmonia', '430957': 'Herveiras', '430960': 'Horizontina', '430965': 'Hulha Negra', '430970': 'Humaitá', '430975': 'Ibarama', '430980': 'Ibiaçá', '430990': 'Ibiraiaras', '430995': 'Ibirapuitã', '431000': 'Ibirubá', '431010': 'Igrejinha', '431020': 'Ijuí', '431030': 'Ilópolis', '431033': 'Imbé', '431036': 'Imigrante', '431040': 'Independência', '431041': 'Inhacorá', '431043': 'Ipê', '431046': 'Ipiranga do Sul', '431050': 'Iraí', '431053': 'Itaara', '431055': 'Itacurubi', '431057': 'Itapuca', '431060': 'Itaqui', '431065': 'Itati', '431070': 'Itatiba do Sul', '431075': 'Ivorá', '431080': 'Ivoti', '431085': 'Jaboticaba', '431087': 'Jacuizinho', '431090': 'Jacutinga', '431100': 'Jaguarão', '431110': 'Jaguari', '431112': 'Jaquirana', '431113': 'Jari', '431115': 'Jóia', '431120': 'Júlio de Castilhos', '431123': 'Lagoa Bonita do Sul', '431125': 'Lagoão', '431127': 'Lagoa dos Três Cantos', '431130': 'Lagoa Vermelha', '431140': 'Lajeado', '431142': 'Lajeado do Bugre', '431150': 'Lavras do Sul', '431160': 'Liberato Salzano', '431162': 'Lindolfo Collor', '431164': 'Linha Nova', '431170': 'Machadinho', '431171': 'Maçambará', '431173': 'Mampituba', '431175': 'Manoel Viana', '431177': 'Maquiné', '431179': 'Maratá', '431180': 'Marau', '431190': 'Marcelino Ramos', '431198': 'Mariana Pimentel', '431200': 'Mariano Moro', '431205': 'Marques de Souza', '431210': 'Mata', '431213': 'Mato Castelhano', '431215': 'Mato Leitão', '431217': 'Mato Queimado', '431220': 'Maximiliano de Almeida', '431225': 'Minas do Leão', '431230': 'Miraguaí', '431235': 'Montauri', '431237': 'Monte Alegre dos Campos', '431238': 'Monte Belo do Sul', '431240': 'Montenegro', '431242': 'Mormaço', '431244': 'Morrinhos do Sul', '431245': 'Morro Redondo', '431247': 'Morro Reuter', '431250': 'Mostardas', '431260': 'Muçum', '431261': 'Muitos Capões', '431262': 'Muliterno', '431265': 'Não-Me-Toque', '431267': 'Nicolau Vergueiro', '431270': 'Nonoai', '431275': 'Nova Alvorada', '431280': 'Nova Araçá', '431290': 'Nova Bassano', '431295': 'Nova Boa Vista', '431300': 'Nova Bréscia', '431301': 'Nova Candelária', '431303': 'Nova Esperança do Sul', '431306': 'Nova Hartz', '431308': 'Nova Pádua', '431310': 'Nova Palma', '431320': 'Nova Petrópolis', '431330': 'Nova Prata', '431333': 'Nova Ramada', '431335': 'Nova Roma do Sul', '431337': 'Nova Santa Rita', '431339': 'Novo Cabrais', '431340': 'Novo Hamburgo', '431342': 'Novo Machado', '431344': 'Novo Tiradentes', '431346': 'Novo Xingu', '431349': 'Novo Barreiro', '431350': 'Osório', '431360': 'Paim Filho', '431365': 'Palmares do Sul', '431370': 'Palmeira das Missões', '431380': 'Palmitinho', '431390': 'Panambi', '431395': 'Pantano Grande', '431400': 'Paraí', '431402': 'Paraíso do Sul', '431403': 'Pareci Novo', '431405': 'Parobé', '431406': 'Passa Sete', '431407': 'Passo do Sobrado', '431410': 'Passo Fundo', '431413': 'Paulo Bento', '431415': 'Paverama', '431417': 'Pedras Altas', '431420': 'Pedro Osório', '431430': 'Pejuçara', '431440': 'Pelotas', '431442': 'Picada Café', '431445': 'Pinhal', '431446': 'Pinhal da Serra', '431447': 'Pinhal Grande', '431449': 'Pinheirinho do Vale', '431450': 'Pinheiro Machado', '431454': 'Pinto Bandeira', '431455': 'Pirapó', '431460': 'Piratini', '431470': 'Planalto', '431475': 'Poço das Antas', '431477': 'Pontão', '431478': 'Ponte Preta', '431480': 'Portão', '431490': 'Porto Alegre', '431500': 'Porto Lucena', '431505': 'Porto Mauá', '431507': 'Porto Vera Cruz', '431510': 'Porto Xavier', '431513': 'Pouso Novo', '431514': 'Presidente Lucena', '431515': 'Progresso', '431517': 'Protásio Alves', '431520': 'Putinga', '431530': 'Quaraí', '431531': 'Quatro Irmãos', '431532': 'Quevedos', '431535': 'Quinze de Novembro', '431540': 'Redentora', '431545': 'Relvado', '431550': 'Restinga Seca', '431555': 'Rio dos Índios', '431560': 'Rio Grande', '431570': 'Rio Pardo', '431575': 'Riozinho', '431580': 'Roca Sales', '431590': 'Rodeio Bonito', '431595': 'Rolador', '431600': 'Rolante', '431610': 'Ronda Alta', '431620': 'Rondinha', '431630': 'Roque Gonzales', '431640': 'Rosário do Sul', '431642': 'Sagrada Família', '431643': 'Saldanha Marinho', '431645': 'Salto do Jacuí', '431647': 'Salvador das Missões', '431650': 'Salvador do Sul', '431660': 'Sananduva', '431670': 'Santa Bárbara do Sul', '431673': 'Santa Cecília do Sul', '431675': 'Santa Clara do Sul', '431680': 'Santa Cruz do Sul', '431690': 'Santa Maria', '431695': 'Santa Maria do Herval', '431697': 'Santa Margarida do Sul', '431700': 'Santana da Boa Vista', '431710': 'SantAna do Livramento', '431720': 'Santa Rosa', '431725': 'Santa Tereza', '431730': 'Santa Vitória do Palmar', '431740': 'Santiago', '431750': 'Santo Ângelo', '431755': 'Santo Antônio do Palma', '431760': 'Santo Antônio da Patrulha', '431770': 'Santo Antônio das Missões', '431775': 'Santo Antônio do Planalto', '431780': 'Santo Augusto', '431790': 'Santo Cristo', '431795': 'Santo Expedito do Sul', '431800': 'São Borja', '431805': 'São Domingos do Sul', '431810': 'São Francisco de Assis', '431820': 'São Francisco de Paula', '431830': 'São Gabriel', '431840': 'São Jerônimo', '431842': 'São João da Urtiga', '431843': 'São João do Polêsine', '431844': 'São Jorge', '431845': 'São José das Missões', '431846': 'São José do Herval', '431848': 'São José do Hortêncio', '431849': 'São José do Inhacorá', '431850': 'São José do Norte', '431860': 'São José do Ouro', '431861': 'São José do Sul', '431862': 'São José dos Ausentes', '431870': 'São Leopoldo', '431880': 'São Lourenço do Sul', '431890': 'São Luiz Gonzaga', '431900': 'São Marcos', '431910': 'São Martinho', '431912': 'São Martinho da Serra', '431915': 'São Miguel das Missões', '431920': 'São Nicolau', '431930': 'São Paulo das Missões', '431935': 'São Pedro da Serra', '431936': 'São Pedro das Missões', '431937': 'São Pedro do Butiá', '431940': 'São Pedro do Sul', '431950': 'São Sebastião do Caí', '431960': 'São Sepé', '431970': 'São Valentim', '431971': 'São Valentim do Sul', '431973': 'São Valério do Sul', '431975': 'São Vendelino', '431980': 'São Vicente do Sul', '431990': 'Sapiranga', '432000': 'Sapucaia do Sul', '432010': 'Sarandi', '432020': 'Seberi', '432023': 'Sede Nova', '432026': 'Segredo', '432030': 'Selbach', '432032': 'Senador Salgado Filho', '432035': 'Sentinela do Sul', '432040': 'Serafina Corrêa', '432045': 'Sério', '432050': 'Sertão', '432055': 'Sertão Santana', '432057': 'Sete de Setembro', '432060': 'Severiano de Almeida', '432065': 'Silveira Martins', '432067': 'Sinimbu', '432070': 'Sobradinho', '432080': 'Soledade', '432085': 'Tabaí', '432090': 'Tapejara', '432100': 'Tapera', '432110': 'Tapes', '432120': 'Taquara', '432130': 'Taquari', '432132': 'Taquaruçu do Sul', '432135': 'Tavares', '432140': 'Tenente Portela', '432143': 'Terra de Areia', '432145': 'Teutônia', '432146': 'Tio Hugo', '432147': 'Tiradentes do Sul', '432149': 'Toropi', '432150': 'Torres', '432160': 'Tramandaí', '432162': 'Travesseiro', '432163': 'Três Arroios', '432166': 'Três Cachoeiras', '432170': 'Três Coroas', '432180': 'Três de Maio', '432183': 'Três Forquilhas', '432185': 'Três Palmeiras', '432190': 'Três Passos', '432195': 'Trindade do Sul', '432200': 'Triunfo', '432210': 'Tucunduva', '432215': 'Tunas', '432218': 'Tupanci do Sul', '432220': 'Tupanciretã', '432225': 'Tupandi', '432230': 'Tuparendi', '432232': 'Turuçu', '432234': 'Ubiretama', '432235': 'União da Serra', '432237': 'Unistalda', '432240': 'Uruguaiana', '432250': 'Vacaria', '432252': 'Vale Verde', '432253': 'Vale do Sol', '432254': 'Vale Real', '432255': 'Vanini', '432260': 'Venâncio Aires', '432270': 'Vera Cruz', '432280': 'Veranópolis', '432285': 'Vespasiano Correa', '432290': 'Viadutos', '432300': 'Viamão', '432310': 'Vicente Dutra', '432320': 'Victor Graeff', '432330': 'Vila Flores', '432335': 'Vila Lângaro', '432340': 'Vila Maria', '432345': 'Vila Nova do Sul', '432350': 'Vista Alegre', '432360': 'Vista Alegre do Prata', '432370': 'Vista Gaúcha', '432375': 'Vitória das Missões', '432377': 'Westfalia', '432380': 'Xangri-lá'}

#A partir de um dataframe retorna o mesmo com identificadores das possiveis duplicidades.
def dupli_df(df, name, date, mother=None, mun = None):

  ################################################# NESTED FUNCTIONS #################################################




  def merge_lists_ratios(data,h_th = 100, l_th = 90):

    '''
    Função para agrupar sublistas dentro de uma lista de dados com base no fuzz.ratio.

    Args:
        data (list): Lista de dados contendo sublistas com indexes e o fuzz.Ratio. ex: [index,RATIO,index]
        h_th (int): Limite superior para o valor na posição 1 de cada sublista. Padrão: 100.
        l_th (int): Limite inferior para o valor na posição 1 de cada sublista. Padrão: 90.

    Returns:
        list: Lista de sublistas agrupadas pelos indexes.
    '''
    # Filtrar sublistas com base nos thresholds de ratio passados.
    data = [x[::2] for x in data if (h_th >= x[1] >= l_th)]

    # Create a dictionary to store the connections between values and sublists
    value_to_sublists = defaultdict(list)

    # Iterate over each sublist in the data
    for sublist in data:
        # Iterate over each value in the sublist
        for value in sublist:
            # Add the current sublist to the list of sublists associated with the value
            value_to_sublists[value].append(sublist)

    # Function to merge sublists and remove duplicate values
    def merge_sublists(sublists):
        merged_list = []
        for sublist in sublists:
            for value in sublist:
                if value not in merged_list:
                    merged_list.append(value)
        return merged_list

    # Perform the merging of sublists
    merged_lists = []
    visited = set()

    # Iterate over each sublist in the data
    for sublist in data:
        # If the current sublist has been visited, continue to the next sublist
        if tuple(sublist) in visited:
            continue

        # Perform a breadth-first search to find all connected sublists
        queue = [sublist]
        connected_sublists = []
        while queue:
            current = queue.pop(0)
            if current not in connected_sublists:
                connected_sublists.append(current)
                visited.add(tuple(current))
                for value in current:
                    if value in value_to_sublists:
                        queue.extend(value_to_sublists[value])

        # Merge the connected sublists and remove duplicate values within the merged list
        merged_list = merge_sublists(connected_sublists)
        merged_lists.append(merged_list)

    return merged_lists



  def merge_lists(data):
    # Create a dictionary to store the connections between values and sublists
    value_to_sublists = defaultdict(list)

    # Iterate over each sublist in the data
    for sublist in data:
        # Iterate over each value in the sublist
        for value in sublist:
            # Add the current sublist to the list of sublists associated with the value
            value_to_sublists[value].append(sublist)



    # Function to merge sublists and remove duplicate values
    def merge_sublists(sublists):
        merged_list = []
        for sublist in sublists:
            for value in sublist:
                if value not in merged_list:
                    merged_list.append(value)
        return merged_list

    # Perform the merging of sublists
    merged_lists = []
    visited = set()

    # Iterate over each sublist in the data
    for sublist in data:
        # If the current sublist has been visited, continue to the next sublist
        if tuple(sublist) in visited:
            continue

        # Perform a breadth-first search to find all connected sublists
        queue = [sublist]
        connected_sublists = []
        while queue:
            current = queue.pop(0)
            if current not in connected_sublists:
                connected_sublists.append(current)
                visited.add(tuple(current))
                for value in current:
                    if value in value_to_sublists:
                        queue.extend(value_to_sublists[value])

        # Merge the connected sublists and remove duplicate values within the merged list
        merged_list = merge_sublists(connected_sublists)
        merged_lists.append(merged_list)

    return merged_lists



  #Gera uma lista deduplicada de matches pela coluna passada
  def match_list(df ,column_name, min_similarity):
    """
    Função para encontrar strings semelhantes em um DataFrame e agrupar seus índices.

    Args:
        df (DataFrame): DataFrame contendo as strings a serem comparadas.
        column_name (str): Nome da coluna no DataFrame contendo as strings.
        min_similarity (float): Valor mínimo de similaridade para considerar uma correspondência.

    Returns:
        list: Lista de listas contendo os índices agrupados de strings semelhantes.

    """
    #Converter a coluna específica do DataFrame em uma lista de strings
    df_list = df[name].apply(str)

    # Realiza a comparação entre as strings e retornar um DataFrame com as correspondências
    matches = match_strings(df_list, n_blocks = 'auto', min_similarity = min_similarity )

    #Remover comparações de linhas consigo mesmas
    matches = matches[matches['left_index'] != matches['right_index']]

    #Criar uma lista de listas com os índices das comparações entre strings
    match_list = []
    match_index = []
    for index in range(len(matches.index)):

      lr_index = [matches.iloc[index]['left_index'], matches.iloc[index]['right_index']]

      match_list.append(lr_index)

    #Agrupar os índices das strings semelhantes
    return merge_lists(match_list)

  print('Carregando....')


  # Cria uma nested list agrupando nomes similares
  match_names_list = match_list(df, name, min_similarity)

  # Loop pelos matches (grupos de nomes similares)

  #Inicia o identificador
  identifier = 0

  # Inicializa uma lista vazia para armazenar as ratios de similaridade
  ratios = []


  for i in range(len(match_names_list)):
    loading = round(i*(100/len(match_names_list)))
    print(loading*'|'+ ' ' + str(loading) +'%')
    matches = df.loc[match_names_list[i]]

    # Reinicializa a lista vazia para armazenar as ratios de similaridade
    ratios = []
    for nm in matches[name]:
      for nm2 in matches[name]:
        Str1 = str(nm)
        Str2 = str(nm2)
        Ratio = fuzz.ratio(Str1.lower(),Str2.lower())

        # Verifica se as strings são nulas ('nan') e atribui uma razão de similaridade de 80 para não exclui-las diretamente da comparação e serem filtradas posteriormente.
        if (Str1 == 'nan' or Str2 == 'nan'):
          Ratio = 80

        ratio_comp = [Str1,Ratio,Str2]
        ratios.append(ratio_comp)

    # Aplica um filtro de valores nas razões de similaridade
    ratios90 = merge_lists_ratios(ratios,h_th = 100, l_th = 90)

    #Cria lista de matches
    for sublist in ratios90:
      matches = matches.loc[matches[name].isin(sublist)]

    # A partir deste ponto, matches contém apenas as linhas que correspondem a nomes similares



    ################################################################### LOCAL E NOME DA MÃE ###################################################################
      if (mother and mun):

        #caso os matches tenham mães não exatas
        if len(matches[mother].unique()) > 1:

          #compara similaridade entre mães com similaridade minima definida
          Ratios = []
          for nm_mae in matches[mother]:
            for nm_mae2 in matches[mother]:
              Str1 = str(nm_mae)
              Str2 = str(nm_mae2)
              Ratio = fuzz.ratio(Str1.lower(),Str2.lower())

              #print(Str1+' '+str(Ratio)+' '+Str2)
              if (Str1 == 'nan' or Str2 == 'nan'):
                Ratio = 80

              ratio_comp = [Str1,Ratio,Str2]
              ratios.append(ratio_comp)

          # Aplica um filtro de valores nas ratios de similaridade para mães.
          ratios90 = merge_lists_ratios(ratios,h_th = 100, l_th = 90)
          ratios80 = merge_lists_ratios(ratios,h_th = 89, l_th = 79)

          #Mesmo identificador para nomes de mãe muito similares
          for sublist in ratios90:
            select90 = matches.loc[matches[mother].isin(sublist)]
            df.loc[select90.index, 'IDENTIFICADOR'] = identifier
            identifier += 1

          #Filtro adicional por data para nomes de mãe pouco similares
          for sublist in ratios80:
            select80 = matches.loc[matches[mother].isin(sublist)]
            #diferencia strings similares com datas diferentes
            ratios = []
            ratios_m_d = []
            #print('--------------------------------------')
            for dat_nasc0 in select80[date].unique():
              for dat_nasc1 in select80[date].unique():
                Str1 = str(dat_nasc0)[:10]
                Str2 = str(dat_nasc1)[:10]
                Ratio = fuzz.token_sort_ratio(Str1.lower(),Str2.lower())
                #print(Str1+' '+str(Ratio)+' '+Str2)
                ratio_comp = [Str1,Ratio,Str2]
                #if ratio_comp[::-1] not in ratios:
                ratios.append(ratio_comp)

            ratios90 = merge_lists_ratios(ratios,h_th = 100, l_th = 90)
            ratios80 = merge_lists_ratios(ratios,h_th = 89, l_th = 80)


            for sublist in ratios90:
              select90 = matches.loc[matches[date].isin(sublist)]
              ratios_m_d.append(select90.index.values.tolist())

            ratios = []
            #Filtra nomes de mãe similares com troca de ordem de nomes
            for nm_mae in select80[mother].unique():
              for nm_mae2 in select80[mother].unique():
                Str1 = str(nm_mae)
                Str2 = str(nm_mae2)
                Ratio = fuzz.partial_token_sort_ratio(Str1.lower(),Str2.lower())
                #print(Str1+' '+str(Ratio)+' '+Str2)
                ratio_comp = [Str1,Ratio,Str2]
                #if ratio_comp[::-1] not in ratios:
                ratios.append(ratio_comp)

            ratios95 = merge_lists_ratios(ratios,h_th = 100, l_th = 95)
            ratios80 = merge_lists_ratios(ratios,h_th = 89, l_th = 80)

            for sublist in ratios95:
              select95 = matches.loc[matches[mother].isin(sublist)]
              ratios_m_d.append(select95.index.values.tolist())

            ratios = []
            #Filtra nomes de RUA similares:
            for mun1 in select80[mun].unique():
              for mun2 in select80[mun].unique():
                Str1 = str(mun)
                Str2 = str(mun2)
                Ratio = fuzz.partial_token_sort_ratio(Str1.lower(),Str2.lower())
                #print(Str1+' '+str(Ratio)+' '+Str2)
                ratio_comp = [Str1,Ratio,Str2]
                #if ratio_comp[::-1] not in ratios:
                ratios.append(ratio_comp)

            ratios90 = merge_lists_ratios(ratios,h_th = 100, l_th = 90)
            ratios80 = merge_lists_ratios(ratios,h_th = 89, l_th = 80)

            for sublist in ratios90:
              select90 = matches.loc[matches[mun].isin(sublist)]
              ratios_m_d.append(select90.index.values.tolist())

            ratios_m_d_dedup = merge_lists(ratios_m_d)
            for sublist in ratios_m_d_dedup:
              df.loc[sublist, 'IDENTIFICADOR'] = identifier
              identifier += 1


        else:
          #Marca matches com números mesmos identificadores
          df.loc[matches.index, 'IDENTIFICADOR'] = identifier
          identifier += 1

    ################################################################### APENAS NOME DA MÃE ###################################################################
      if (mother and not mun):
        #caso os matches tenham mães não exatas e nenhuma faltando
        if len(matches[mother].unique()) > 1:
          #compara similaridade entre mães com similaridade minima definida
          Ratios = []
          for nm_mae in matches[mother]:
            for nm_mae2 in matches[mother]:
              Str1 = str(nm_mae)
              Str2 = str(nm_mae2)
              Ratio = fuzz.ratio(Str1.lower(),Str2.lower())

              if (Str1 == 'nan' or Str2 == 'nan'):
                Ratio = 80

              ratio_comp = [Str1,Ratio,Str2]

              ratios.append(ratio_comp)

          ratios90 = merge_lists_ratios(ratios,h_th = 100, l_th = 90)
          ratios80 = merge_lists_ratios(ratios,h_th = 89, l_th = 70)

          #Mesmo identificador para nomes de mãe muito similares
          for sublist in ratios90:
            select90 = matches.loc[matches[mother].isin(sublist)]
            df.loc[select90.index, 'IDENTIFICADOR'] = identifier
            identifier += 1

          #Filtro adicional por data para nomes de mãe pouco similares
          for sublist in ratios80:
            select80 = matches.loc[matches[mother].isin(sublist)]
            #diferencia strings similares com datas diferentes
            ratios = []
            ratios_m_d = []
            #print('--------------------------------------')
            for dat_nasc0 in select80[date].unique():
              for dat_nasc1 in select80[date].unique():
                Str1 = str(dat_nasc0)[:10]
                Str2 = str(dat_nasc1)[:10]
                Ratio = fuzz.token_sort_ratio(Str1.lower(),Str2.lower())
                #print(Str1+' '+str(Ratio)+' '+Str2)
                ratio_comp = [Str1,Ratio,Str2]
                #if ratio_comp[::-1] not in ratios:
                ratios.append(ratio_comp)

            ratios90 = merge_lists_ratios(ratios,h_th = 100, l_th = 90)
            ratios80 = merge_lists_ratios(ratios,h_th = 89, l_th = 80)


            for sublist in ratios90:
              select90 = matches.loc[matches[date].isin(sublist)]
              ratios_m_d.append(select90.index.values.tolist())

            ratios = []
            #Filtra nomes de mãe similares com troca de ordem de nomes
            for nm_mae in select80[mother].unique():
              for nm_mae2 in select80[mother].unique():
                Str1 = str(nm_mae)
                Str2 = str(nm_mae2)
                Ratio = fuzz.partial_token_sort_ratio(Str1.lower(),Str2.lower())
                #print(Str1+' '+str(Ratio)+' '+Str2)
                ratio_comp = [Str1,Ratio,Str2]
                #if ratio_comp[::-1] not in ratios:
                ratios.append(ratio_comp)

            ratios95 = merge_lists_ratios(ratios,h_th = 100, l_th = 95)
            ratios80 = merge_lists_ratios(ratios,h_th = 89, l_th = 80)

            for sublist in ratios95:
              select95 = matches.loc[matches[mother].isin(sublist)]
              ratios_m_d.append(select95.index.values.tolist())

            ratios_m_d_dedup = merge_lists(ratios_m_d)
            for sublist in ratios_m_d_dedup:
              df.loc[sublist, 'IDENTIFICADOR'] = identifier
              identifier += 1


        else:
          #Marca matches com números mesmos identificadores
          df.loc[matches.index, 'IDENTIFICADOR'] = identifier
          identifier += 1

    ################################################################### APENAS LOCAL ###################################################################
      if (mun and not mother):

          #diferencia strings similares com datas diferentes
          if (len(matches[date].unique())) > 1:
            ratios = []
            #print('--------------------------------------')
            for dat_nasc0 in matches[date].unique():
              for dat_nasc1 in matches[date].unique():
                Str1 = dat_nasc0[:10]
                Str2 = dat_nasc1[:10]
                Ratio = fuzz.token_sort_ratio(Str1.lower(),Str2.lower())

                if (Str1 == 'nan' or Str2 == 'nan'):
                  Ratio = 80

                ratio_comp = [Str1,Ratio,Str2]
                ratios.append(ratio_comp)

            ratios90 = merge_lists_ratios(ratios,h_th = 100, l_th = 90)
            ratios80 = merge_lists_ratios(ratios,h_th = 89, l_th = 80)

            for sublist in ratios90:
              select90 = matches.loc[matches[date].isin(sublist)]
              df.loc[select90.index, 'IDENTIFICADOR'] = identifier
              identifier += 1

            for sublist in ratios80:
              select80 = matches.loc[matches[date].isin(sublist)]
              if len(df[mun].loc[select80.index].unique()) == 1:
                df.loc[select80.index, 'IDENTIFICADOR'] = identifier
                identifier += 1

          else:
            #Marca duplicatas com números identificadores
            df.loc[matches.index, 'IDENTIFICADOR'] = identifier
            identifier += 1

      clear_output()
#Marca linhas com strings que não possuiram matches com numeros identifcadores únicos
  for index, line in df.iterrows():
    if np.isnan(df['IDENTIFICADOR'].iloc[index]):
      df['IDENTIFICADOR'].iloc[index] = identifier
      identifier +=1
#Retorna df com identificadores de duplicatas
  clear_output()
  print("##############################CONCLUIDO##############################")
  return df.sort_values(by = name)

#remove acentos para facilitar comparação entre strings
def remove_accents(a):
    return unidecode.unidecode(a)


In [ ]:
#@title GERAR IDENTIFICAÇÃO DE DUPLICIDADE

if mother:
  df[mother] = df[mother].apply(str)
  df[mother] = df[mother].apply(remove_accents)

if mun:
  df[mun] = df[mun].apply(str).map(dict_id_mun).fillna(df[mun])
  df[mun] = df[mun].apply(str).apply(remove_accents)



df_dupli = dupli_df(df,name, date, mother, mun)

df_dupli['nomes'] = df_dupli[name]

if mother:
  df_dupli['nomes_mae'] = df_dupli[mother]


  df_dupli['data_nasc'] = df_dupli[date]

if mun:
  df_dupli['local'] = df_dupli[mun]



df_dupli.sort_values(by = name).head(500)

In [ ]:
#@title Download Duplicidades
donwload_name = input('Nome do arquivo: ')

df_dupli.to_excel(donwload_name + str(dt.today()) + '.xlsx', engine='xlsxwriter')

files.download(donwload_name + str(dt.today()) + '.xlsx')